# Apprentissage Random Forest avec API SWAT avec Python

Ce pipeline permet d'enchaîner l'apprentissage (sur 400 arbres et une profondeur de 13 maximum), de générer le fichier astore de scoring pour mise en production directe sur la plateforme SAS9.4M5. L'exécution a été effectuée sur un dataset d'environ 500 000 lignes et 200 variables avec une distribution équivalente de celle partagée.

Un apprentissage avec autotuning a été lancé afin de trouver une combinatoire d'hyperparamètre plus optimale. Cette autotuning a permis en 900s environ d'obtenir les hypeparamètres suivants pour la random forest : 

Parameter                        Name              Value
0                          Evaluation  Evaluation     31
1                     Number of Trees       NTREE    111
2          Number of Variables to Try           M     46
3                           Bootstrap   BOOTSTRAP    0.1
4                 Maximum Tree Levels    MAXLEVEL     29
5  Misclassification Error Percentage   Objective  12.40

Chaque sortie de chaque étape a été gardée dans le notebook pour constater les temps de traitement et les tailles des fichiers produits.

Par exemple : l'apprentissage de la foret sur 500000 lignes et 200 variables prends 78 secondes sur une plateforme AWS équivalente à 16 coeurs physique et 128 Go de RAM.

### La documentation générale sur la programmation en Viya en SAS/Python/R est située ici : 
http://go.documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=9.4_3.4&docsetId=pgmsashome&docsetTarget=home.htm&locale=en

### La documentation de l'API SWAT Python pour Viya est située ici :  
https://sassoftware.github.io/python-swat/



In [3]:
import os
import pandas as pd
import swat
import sys
from sasctl import Session

In [21]:
target          = "CIBLE"
class_inputs    = ["var1","var21","var37","var38","var89","var90","var91","var103","var163","var165","var166","var167","var168","var169","var170","var171","var172","var173","var174","var175","var176","var177","var180","var182","var183","var184","var185","var186","var187","var188","var189","var190","var191","var192","var193","var194","var195","var196"]
class_vars      = [target] + class_inputs
interval_inputs = ["var3","var4","var5","var6","var7","var8","var9","var10","var11","var12","var13","var14","var15","var16","var17","var18","var19","var20","var22","var23","var24","var25","var26","var27","var28","var29","var30","var31","var32","var33","var34","var35","var36","var39","var40","var41","var42","var43","var44","var45","var46","var47","var48","var49","var50","var51","var52","var53","var54","var55","var56","var57","var58","var59","var60","var61","var62","var63","var64","var65","var66","var67","var68","var69","var70","var71","var72","var73","var74","var75","var76","var77","var78","var79","var80","var81","var82","var83","var84","var85","var86","var87","var88","var92","var93","var94","var95","var96","var97","var98","var99","var100","var101","var102","var104","var105","var106","var107","var108","var109","var110","var111","var112","var113","var114","var115","var116","var117","var118","var119","var120","var121","var122","var123","var124","var125","var126","var127","var128","var129","var130","var131","var132","var133","var134","var135","var136","var137","var138","var139","var140","var141","var142","var143","var144","var145","var146","var147","var148","var149","var150","var151","var152","var153","var154","var155","var156","var157","var158","var159","var160","var161","var162","var164","var178","var179","var181"]
all_inputs      = interval_inputs + class_inputs

indata = 'train_sample_retail_banking'


In [9]:
cashost='frasepviya34.aws.sas.com'
casport=5570
sess = swat.CAS(cashost, casport)

# Load the needed action sets for this example:
sess.loadactionset('datapreprocess')
sess.loadactionset('sampling')
sess.loadactionset('decisiontree')
sess.loadactionset('astore')
sess.loadactionset('autotune')

NOTE: Added action set 'datapreprocess'.
NOTE: Added action set 'sampling'.
NOTE: Added action set 'decisiontree'.
NOTE: Added action set 'astore'.
NOTE: Added action set 'autotune'.


[actionset]

 'autotune'

+ Elapsed: 0.000873s, user: 0.000239s, sys: 0.000618s, mem: 0.218mb

In [4]:
# On récupère l'aide en ligne sur les actions (par actionset) disponibles avec l'API avec les actionsset chargés précedemment

sess.help()

NOTE: Available Action Sets and Actions:
NOTE:    accessControl
NOTE:       accessPersonalCaslibs - Provides administrative access to all personal caslibs (CASUSER and CASUSERHDFS)
NOTE:       assumeRole - Assumes a role
NOTE:       dropRole - Relinquishes a role
NOTE:       showRolesIn - Shows the currently active role
NOTE:       showRolesAllowed - Shows the roles that a user is a member of
NOTE:       isInRole - Shows whether a role is assumed
NOTE:       isAuthorized - Shows whether access is authorized
NOTE:       isAuthorizedActions - Shows whether access is authorized to actions
NOTE:       isAuthorizedTables - Shows whether access is authorized to tables
NOTE:       isAuthorizedColumns - Shows whether access is authorized to columns
NOTE:       listAllPrincipals - Lists all principals that have explicit access controls
NOTE:       whatIsEffective - Lists effective access and explanations (Origins)
NOTE:       whatCheckoutsExist - Lists checkouts held on an object, its parents, 

,name,description
0,accessPersonalCaslibs,Provides administrative access to all personal...
1,assumeRole,Assumes a role
2,dropRole,Relinquishes a role
3,showRolesIn,Shows the currently active role
4,showRolesAllowed,Shows the roles that a user is a member of
5,isInRole,Shows whether a role is assumed
6,isAuthorized,Shows whether access is authorized
7,isAuthorizedActions,Shows whether access is authorized to actions
8,isAuthorizedTables,Shows whether access is authorized to tables
9,isAuthorizedColumns,Shows whether access is authorized to columns


In [11]:
# Si la table n'est pas chargée, on peut aussi se charger une version en session privée
out = sess.loadtable(indata+'.sas7bdat', caslib='MYDATA', casout=dict(name=indata, caslib='MYDATA'))

# Déclaration de la table CAS déjà chargée en mémoire et accessible par les utilisateurs Python/SAS/R
casindata = sess.CASTable('TRAIN_SAMPLE_RETAIL_BANKING', caslib='MYDATA')

NOTE: Cloud Analytic Services made the file train_sample_retail_banking.sas7bdat available as table TRAIN_SAMPLE_RETAIL_BANKING in caslib MYDATA.


In [6]:
# On récupère l'aide sur l'action permettant d'échantillonner avec une stratification

sess.help(action="stratified")

NOTE: Information for action 'sampling.stratified':
NOTE: The following parameters are accepted.  Default values are shown.
NOTE:    list table={
NOTE:       specifies the input data table.
NOTE:       string name=NULL (required),
NOTE:       specifies the name of the table to use.
NOTE:       string caslib=NULL,
NOTE:       specifies the caslib that contains the table that you want to use with the action. By default, the active caslib is used. Specify a value only if you need to access a table from a different caslib.
NOTE:       string where=NULL,
NOTE:       specifies an expression for subsetting the input data.
NOTE:       array of groupBy={
NOTE:       specifies the names of the variables to use for grouping results.
NOTE:          string name=NULL (required),
NOTE:       specifies the name for the variable.
NOTE:          string label=NULL,
NOTE:       specifies the descriptive label for the variable.
NOTE:          int32 formattedLength=0,
NOTE:       specifies the length of for

[stratified]

 True

+ Elapsed: 0.00923s, user: 0.00909s, sys: 0.000123s, mem: 0.198mb

In [12]:
# Partitionnement des données sur le serveur CAS et création de la table partitionnée en mémoire

casindata_part = sess.CASTable('train_sample_retail_banking_part', replace=True)

casindata.groupby(by='CIBLE').sampling.stratified(
  output=dict(casout='train_sample_retail_banking_part', copyvars='all'),
  samppct=75,
  partind=True
)

NOTE: Using SEED=1361439120 for sampling.


[outputSize]

 {'outputNObs': 594285.0, 'outputNVars': 214}

[STRAFreq]

 Frequencies
 
    ByGrpID CIBLE    NObs   NSamp
 0        0     0  517797  388348
 1        1     1   76488   57366

[OutputCasTables]

                 casLib                              Name Label    Rows  \
 0  CASUSER(viyademo01)  train_sample_retail_banking_part        594285   
 
    Columns                                           casTable  
 0      214  CASTable('train_sample_retail_banking_part', c...  

+ Elapsed: 1.03s, user: 8.67s, sys: 1.99s, mem: 81.9mb

In [13]:
# On crée une vue sur les données de training
casindata_part_1 = casindata_part.query('_partind_ = 1')


In [14]:
# On lance l'apprentissage de modèle de random forest sur 400 arbres et 13 en profondeur maximum. 
#D'autres paramètres sont mis avec leurs valeurs par défaut

forest_model = sess.CASTable('retail_banking_forest_model', replace=True)

casindata_part_1.decisiontree.foresttrain(
    inputs=all_inputs,  
    nominals=class_vars,
    prune="FALSE",
    alpha=0,
    missing='USEINSEARCH',
    leafsize=5,
    oob="TRUE",
    minuseinsearch=1,
    maxBranch=2,
    target='CIBLE',  
    ntree=400,  
    maxlevel=13,  
    crit='GAINRATIO',
    vote='PROB',
    nbins=20,
    bootstrap=0.6,
    savestate=dict(name='forest_model_state', compress='true', replace='true') #sauvegarde de l'état du modèle pour le déploiement
)

NOTE: Wrote 37282860 bytes to the savestate file forest_model_state.


,Descr,Value
0,Number of Trees,400.000000
1,Number of Selected Variables (M),14.000000
2,Random Number Seed,0.000000
3,Bootstrap Percentage (%),60.000000
4,Number of Bins,20.000000
5,Number of Variables,195.000000
6,Confidence Level for Pruning,0.250000
7,Max Number of Tree Nodes,315.000000
8,Min Number of Tree Nodes,33.000000
9,Max Number of Branches,2.000000


In [15]:
# On lance l'apprentissage de modèle de random forest en réalisant un autotuning (optimisation des paramètres automatique par SAS avec

forest_model_autotune = sess.CASTable('retail_banking_forest_model_autotune', replace=True)

result = sess.autotune.tuneForest( 
    trainOptions=
            dict(
                table=dict(name="train_sample_retail_banking_part",where="_partind_ = 1"),
                inputs=all_inputs,
                target="CIBLE",
                nominals=class_vars,
                casOut=dict(name="retail_banking_forest_model_autotune",replace="True"),
                savestate=dict(name='forest_model_state_autotune', compress='true', replace='true')
             ),
             tunerOptions=dict(seed=54321)
     )


NOTE: Autotune is started for 'Forest' model.
NOTE: Autotune option SEARCHMETHOD='GA'.
NOTE: Autotune option MAXTIME=36000 (sec.).
NOTE: Autotune option SEED=54321.
NOTE: Autotune objective is 'Misclassification Error Percentage'.
NOTE: Autotune number of parallel evaluations is set to 4, each using 0 worker nodes.
         Iteration       Evals     Best Objective  Elapsed Time
                 0           1             12.783         44.68
                 1          19             12.495        620.32
                 2          35              12.48       1384.60
                 3          50             12.465       1955.00
                 4          65             12.465       2413.32
                 5          79             12.459       2927.83
NOTE: Data was partitioned during tuning, to tune based on validation score; the final model is trained and scored on all data.
NOTE: Wrote 16501732 bytes to the savestate file forest_model_state_autotune.
NOTE: Autotune time is 2973.0

In [16]:
# The following commands display the tables that are produced by this action call:

print(result.TunerInfo)
print(result.TunerResults)
print(result.IterationHistory)
print(result.EvaluationHistory)
print(result.BestConfiguration)
print(result.TunerSummary)
print(result.TunerTiming)
print(result.TunerCasOutputTables)

Tuner Information

                           Parameter                               Value
0                         Model Type                              Forest
1           Tuner Objective Function  Misclassification Error Percentage
2                      Search Method                                  GA
3                    Population Size                                  10
4                 Maximum Iterations                                   5
5     Maximum Tuning Time in Seconds                               36000
6                    Validation Type                    Single Partition
7      Validation Partition Fraction                                0.30
8                          Log Level                                   2
9                               Seed                               54321
10    Number of Parallel Evaluations                                   4
11  Number of Workers per Subsession                                   0
Tuner Results

    Evaluation  N

In [18]:
# On sérialise le modèle choisi produit sur disque pour le déploiement sur le serveur batch de production

rfstore = sess.astore.download(rstore='forest_model_state')

path = '/opt/shared/astore/'
file_name = os.path.join(path, indata+'.sasast')
with open(file_name, 'wb') as file:
    file.write(rfstore['blob'])
    
rfstoretuned = sess.astore.download(rstore='forest_model_state_autotune')
file_name = os.path.join(path, indata+'_tuned.sasast')
with open(file_name, 'wb') as file:
    file.write(rfstoretuned['blob'])

In [19]:
# This is the same as sess.endsession(); sess.close();
sess.terminate()